This solution is based on the notebook by IlyaGusev:
https://github.com/IlyaGusev/nlp-practice/blob/master/rupos.ipynb

In [1]:
from tqdm import tqdm

In [2]:
# Имена файлов с данными.
from os.path import join
PATH_DATA = 'data'
TRAIN_FILENAME = join(PATH_DATA, 'train.csv')
TEST_FILENAME = join(PATH_DATA, 'test.csv')

In [3]:
# Считывание файлов.
from collections import namedtuple
WordForm = namedtuple('WordForm', 'word pos gram')

def get_sentences(filename, is_train):
    with open(filename, 'r', encoding='utf-8') as r:
        sentence = []
        for line in r:
            line = line.strip()
            if len(line) == 0:
                if len(sentence) > 0:
                    yield sentence
                    sentence = []
                continue
            if is_train:
                line_parts = line.split('\t')
                word = line_parts[2]
                pos, gram = line_parts[3].split('#')
                sentence.append(WordForm(word, pos, gram))
            else:
                word = line.split('\t')[2]
                sentence.append(word)
        if len(sentence) != 0:
            yield sentence

In [4]:
train = list(get_sentences(TRAIN_FILENAME, True))
test = list(get_sentences(TEST_FILENAME, False))

In [5]:
# Класс для удобной векторизации грамматических значений.
import jsonpickle
import os
from collections import defaultdict
from typing import Dict, List, Set

def process_gram_tag(gram: str):
    gram = gram.strip().split('|')
    return '|'.join(sorted(gram))


def get_empty_category():
    return {GrammemeVectorizer.UNKNOWN_VALUE}


class GrammemeVectorizer(object):
    UNKNOWN_VALUE = 'Unknown'

    def __init__(self, dump_filename: str):
        self.all_grammemes = defaultdict(get_empty_category)  # type: Dict[str, Set]
        self.vectors = []  # type: List[List[int]]
        self.name_to_index = {}  # type: Dict[str, int]
        self.dump_filename = dump_filename  # type: str
        if os.path.exists(self.dump_filename):
            self.load()

    def add_grammemes(self, pos_tag: str, gram: str) -> int:
        gram = process_gram_tag(gram)
        vector_name = pos_tag + '#' + gram
        if vector_name not in self.name_to_index:
            self.name_to_index[vector_name] = len(self.name_to_index)
            self.all_grammemes['POS'].add(pos_tag)
            gram = gram.split('|') if gram != '_' else []
            for grammeme in gram:
                category, value = grammeme.split('=')
                self.all_grammemes[category].add(value)
        return self.name_to_index[vector_name]

    def init_possible_vectors(self) -> None:
        self.vectors = []
        for grammar_val, index in sorted(self.name_to_index.items(), key=lambda x: x[1]):
            pos_tag, grammemes = grammar_val.split('#')
            grammemes = grammemes.split('|') if grammemes != '_' else []
            vector = self.__build_vector(pos_tag, grammemes)
            self.vectors.append(vector)

    def get_vector(self, vector_name: str) -> List[int]:
        if vector_name not in self.name_to_index:
            return [0] * len(self.vectors[0])
        return self.vectors[self.name_to_index[vector_name]]

    def get_vector_by_index(self, index: int) -> List[int]:
        return self.vectors[index] if 0 <= index < len(self.vectors) else [0] * len(self.vectors[0])

    def get_ordered_grammemes(self) -> List[str]:
        flat = []
        sorted_grammemes = sorted(self.all_grammemes.items(), key=lambda x: x[0])
        for category, values in sorted_grammemes:
            for value in sorted(list(values)):
                flat.append(category + '=' + value)
        return flat
    
    def save(self) -> None:
        with open(self.dump_filename, 'w') as f:
            f.write(jsonpickle.encode(self, f))

    def load(self):
        with open(self.dump_filename, 'r') as f:
            vectorizer = jsonpickle.decode(f.read())
            self.__dict__.update(vectorizer.__dict__)

    def size(self) -> int:
        return len(self.vectors)

    def grammemes_count(self) -> int:
        return len(self.get_ordered_grammemes())

    def is_empty(self) -> int:
        return len(self.vectors) == 0

    def get_name_by_index(self, index):
        d = {index: name for name, index in self.name_to_index.items()}
        return d[index]

    def get_index_by_name(self, name):
        pos = name.split('#')[0]
        gram = process_gram_tag(name.split('#')[1])
        return self.name_to_index[pos + '#' + gram]

    def __build_vector(self, pos_tag: str, grammemes: List[str]) -> List[int]:
        vector = []
        gram_tags = {pair.split('=')[0]: pair.split('=')[1] for pair in grammemes}
        gram_tags['POS'] = pos_tag
        sorted_grammemes = sorted(self.all_grammemes.items(), key=lambda x: x[0])
        for category, values in sorted_grammemes:
            value_correct = gram_tags[category] if category in gram_tags else GrammemeVectorizer.UNKNOWN_VALUE
            vector.extend(1 if value == value_correct else 0 for value in sorted(list(values)))
        return vector

In [6]:
from pymorphy2 import MorphAnalyzer
from russian_tagsets import converters

morph = MorphAnalyzer()
to_ud = converters.converter('opencorpora-int', 'ud14')

def convert_from_opencorpora_tag(tag, text):
    ud_tag = to_ud(str(tag), text)
    pos, gram = ud_tag.split()
    return pos, gram

def fill_all_variants(word, vectorizer):
    for parse in morph.parse(word):
        pos, gram = convert_from_opencorpora_tag(parse.tag, parse.word)
        vectorizer.add_grammemes(pos, gram)

vectorizer = GrammemeVectorizer('vectorizer.json')
if vectorizer.is_empty():
    print('Add train sentences to vectorizer')
    for sentence in tqdm(train):
        for form in sentence:
            fill_all_variants(form.word, vectorizer)
    print('Add test sentences to vectorizer')
    for sentence in tqdm(test):
        for word in sentence:
            fill_all_variants(word, vectorizer)
    print('Init vectors in vectorizer')
    vectorizer.init_possible_vectors()
    vectorizer.save()

In [7]:
vectorizer_output = GrammemeVectorizer('vectorizer_output.json')
if vectorizer_output.is_empty():
    for sentence in tqdm(train):
        for form in sentence:
            vectorizer_output.add_grammemes(form.pos, gram)
    vectorizer_output.init_possible_vectors()
    vectorizer_output.save()

In [8]:
# Получение признаков для конкретного контекста.
def get_context_features(i, parse_sentence, context_len):
    sample = []
    left = i - (context_len - 1) // 2
    right = i + context_len // 2
    if left < 0:
        sample.extend(0 for _ in range(vectorizer.grammemes_count() * (-left)))
    for parse in parse_sentence[max(left, 0): min(right + 1, len(sentence))]:
        pos, gram = convert_from_opencorpora_tag(parse.tag, parse.word)
        gram = process_gram_tag(gram)
        sample.extend(vectorizer.get_vector(pos + '#' + gram))
    if right > len(sentence) - 1:
        sample.extend(0 for _ in range(vectorizer.grammemes_count() * (right - len(sentence) + 1)))
    assert len(sample) == context_len * vectorizer.grammemes_count()
    return sample

In [9]:
# Загрузка обучающей выборки.
import numpy as np
import os

context_len = 5

TRAIN_SAMPLES_PATH = 'samples.npy'
ANSWERS_PATH = 'answers.npy'
if not os.path.exists(TRAIN_SAMPLES_PATH) or not os.path.exists(ANSWERS_PATH):
    n = sum([1 for sentence in train for word in sentence])
    samples = np.zeros((n, context_len * vectorizer.grammemes_count()), dtype='bool_')
    answers = np.zeros((n, ), dtype='int')
    index = 0
    for sentence in tqdm(train):
        parse_sentence = [morph.parse(form.word)[0] for form in sentence]
        for i, form in enumerate(sentence):
            samples[index] = get_context_features(i, parse_sentence, context_len)
            gram = process_gram_tag(form.gram)
            answers[index] = vectorizer_output.get_index_by_name(form.pos + '#' + gram)
            index += 1
    np.save(TRAIN_SAMPLES_PATH, samples)
    np.save(ANSWERS_PATH, answers)
else:
    samples = np.load(TRAIN_SAMPLES_PATH)
    answers = np.load(ANSWERS_PATH)
print(samples[0])
print(answers[0])

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False  True False False  True False False
 False False False False  True False False  True False False False  True
 False False  True False  True False False False  True False False False
  True False False False False False False False Fa

In [10]:
# Выбор классификатора
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=100, subsample=0.5, max_features='sqrt', verbose=2)

In [11]:
# %%time

# # Кросс-валидация
# from sklearn.model_selection import cross_val_score
# X, y = samples[:20000], answers[:20000]

# scores = cross_val_score(clf, X, y, cv=5)
# print(scores)

In [12]:
# Загрузка тестовой выборки
TEST_SAMPLES_PATH = 'test_samples.npy'
ANSWERS_PATH = 'answers.npy'
if not os.path.exists(TEST_SAMPLES_PATH):
    n = sum([1 for sentence in test for word in sentence])
    test_samples = np.zeros((n, context_len * vectorizer.grammemes_count()), dtype='bool_')
    index = 0
    for sentence in tqdm(test):
        parse_sentence = [morph.parse(word)[0] for word in sentence]
        for i, word in enumerate(sentence):
            test_samples[index] = get_context_features(i, parse_sentence, context_len)
            index += 1
    np.save(TEST_SAMPLES_PATH, test_samples)
else:
    test_samples = np.load(TEST_SAMPLES_PATH)

In [13]:
%%time

# Обучение классификатора.
X, y = samples, answers
clf.fit(X, y)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1 1241770617093583081706277375487852441548866426442350592.0000 -1236805383171827736670379958583511400814637804995739648.0000        24304.58m
         2 1238988731126539267398955232089375060583774307158589440.0000           0.0000        24357.85m


KeyboardInterrupt: 

In [14]:
# Предсказания.
answers = []
batch_size = 1000
n_batches = len(test_samples) // batch_size
for i in range(n_batches):
    answers.extend(list(clf.predict(test_samples[i * batch_size: i * batch_size + batch_size])))
answers.extend(list(clf.predict(test_samples[n_batches * batch_size:])))

AttributeError: 'NoneType' object has no attribute 'tree_'

In [ ]:
# Сохранение посылки
with open('subm.csv', 'w') as f: 
    f.write('Id,Prediction\n')
    for index, answer in enumerate(answers):
        f.write(str(index) + ',' + vectorizer_output.get_name_by_index(answer) + '\n')